In [1]:
import urllib.request as urllib2
import pandas as pd
import requests  
from bs4 import BeautifulSoup  
from tqdm import tqdm
import json
from tabula import read_pdf
from tabulate import tabulate
import numpy as np

In [2]:
def search(df: pd.DataFrame, substring: str, case: bool = False) -> pd.DataFrame:
    mask = np.column_stack([df[col].astype(str).str.contains(substring.lower(), case=case, na=False) for col in df])
    return df.loc[mask.any(axis=1)]
case: bool = False
def isNaN(string):
    return string != string

In [3]:
all_data = []

In [4]:
document_names = ['document.pdf','document1.pdf','document11.pdf']

for document_name in document_names:
    
    print(document_name)
    new_dict = {}
    # Reading Table 1-----------------------
    lt_top = 50
    lt_bottom = 150
    step_size = 5
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True,pages = 1) #address of pdf file

    while (search(df[0], 'EIN').shape[0]<1) or (search(df[0].columns.to_frame(), 'Report to').shape[0]<1):  
        df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True,pages = 1) #address of pdf file
        if search(df[0], 'EIN').shape[0]<1:
            lt_bottom += step_size
        if search(df[0].columns.to_frame(), 'Report to').shape[0]<1:
            lt_top += step_size
        df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file

    # Storing EIN and agency name
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom - step_size,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file
    # print(df[0])
    new_dict['Agency'] = df[0][df[0].columns[0]][0]
    new_dict['city_state_PIN'] = df[0][df[0].columns[0]][2]

    substr = 'EIN'
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file
    df1 = search(df[0], substr)
    n_col = np.column_stack([df1[col].astype(str).str.contains(substr.lower(), case=case, na=False) for col in df1])
    n_col = n_col[0].tolist().index(True)
    if ":" in df1[df[0].columns[n_col]][df1[df[0].columns[n_col]].index[0]].lower():
        val_var = df1[df[0].columns[n_col]][df1[df[0].columns[n_col]].index[0]].split(':')[1]
        if  isNaN(val_var):
            val_var = df1.iloc[0,n_col+1]
    new_dict[substr] = val_var.strip()

    # Reading table 2--------------------
    lt_top =  160
    lt_bottom = 300
    step_size = 3
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file
    while (search(df[0].columns.to_frame(), 'StarLiMS').shape[0]<1) or (search(df[0], 'Profile').shape[0]<1):
        df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True,pages = 1) #address of pdf file
        if search(df[0], 'Profile').shape[0]<1:
            lt_bottom += step_size

        if (search(df[0].columns.to_frame(), 'StarLiMS').shape[0]<1) :
            lt_top += step_size
        df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file

    substr = 'StarLiMS'
    val_var = search(df[0].columns.to_frame(),'StarLiMS')[0].values[0].split(':')[1].strip()
    new_dict[substr] = val_var

    # Scrape store variable well / test identification data
    substring = ['Collected','Received','Sampling Point','Well Permit No.','Sample Type','Sample Source','Well Permit Number']
    for substr in substring:
        if search(df[0].columns.to_frame(), substr).shape[0]>0:            
            df1 = df[0].columns.to_frame()
            n_col = df1[np.column_stack([df1[col].astype(str).str.contains(substr.lower(), case=case, na=False) for col in df1])]
            val_var = df[0].iloc[0,df[0].columns.get_loc(n_col[0].values[0])+1]
            new_dict[substr] = val_var
    #         print(substr, val_var)
        else:
            df1 = search(df[0],substr)
            if df1.shape[0]>0:
                n_col = np.column_stack([df1[col].astype(str).str.contains(substr.lower(), case=case, na=False) for col in df1])
                n_col = n_col[0].tolist().index(True)
                if n_col+1>df[0].shape[1]-1: # may be overfitted
                    val_var = df[0][df1.columns[n_col]][df1.index[0]+1]
                else:
                    val_var = df1.iloc[0,n_col+1]
                if  isNaN(val_var):
                    if ":" in df1[df[0].columns[n_col]][df1[df[0].columns[n_col]].index[0]].lower():
                        val_var = df1[df[0].columns[n_col]][df1[df[0].columns[n_col]].index[0]].split(':')[1]
                new_dict[substr] = val_var
    #             print(substr, val_var)

    # Reading table 3------------------------
    lt_top = 300
    lt_bottom = 400
    step_size = 5
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True,pages = 1) #address of pdf file
    while ('Analyte' not in df[0].columns) or (search(df[0], 'Report').shape[0]<1):
        if'Analyte' not in df[0].columns:
            lt_top += step_size
        if search(df[0], 'Report').shape[0]<1:
            lt_bottom += step_size
        df = read_pdf(document_name,area=(lt_top,0,lt_bottom,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file
    df = read_pdf(document_name,area=(lt_top,0,lt_bottom-step_size,550),output_format="dataframe", stream=True, pages = 1) #address of pdf file
#     print(df[0])

    df1 = df[0][['Analyte','Test Result']].transpose()
    df1.columns = df1.iloc[0,:]ce
    col_sel = df1.iloc[0,:]
    df1.reset_index(0,inplace=True)
    test_res_dict = df1.to_dict('records')[1]
    new_dict = new_dict | test_res_dict
    
    # Removing slight difference in names
    if 'Well Permit Number' in new_dict.keys():
        new_dict['Well Permit No.'] = new_dict.pop('Well Permit Number')
    if 'Total Coliform, Colilert' in new_dict.keys():
        new_dict['Total Coliform'] = new_dict.pop('Total Coliform, Colilert')
    if 'E. coli, Colilert' in new_dict.keys():
        new_dict['E. coli'] = new_dict.pop('E. coli, Colilert')
    
    all_data.append(new_dict)

In [42]:
pd.set_option('display.max_columns', 50)
kp = pd.DataFrame(all_data)
kp

,Agency,city_state_PIN,EIN,StarLiMS,Collected,Received,Sampling Point,Well Permit No.,Sample Type,Sample Source,index,Total Coliform,E. coli,Arsenic,Barium,Cadmium,Calcium,Chloride,Chromium,Copper,Fluoride,Iron,Lead,Magnesium,Manganese,Mercury,Nickel,pH,Selenium,Silver,Sodium,Sulfate,Total Alkalinity,Total Hardness,Zinc
0,ALAMANCE CO ENVIRONMENTAL HEALTH,"Burlington, NC 27217",566000271EH,ES230314-0074,03/14/2023,03/14/2023,Well head,9-20-79,,New Well,Test Result,Present,Absent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANSON CO ENVIRONMENTAL HEALTH,"WADESBORO, NC 28170",566000273EH,ES040611-0064001,04/06/2011,NaN,,031,NaN,,Test Result,Absent,Absent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UNION CO ENVIRONMENTAL HEALTH,"MONROE, NC 28112",566000345EH,ES230215-0046,02/15/2023,02/15/2023,Well,NaN,Raw,Well water,Test Result,NaN,NaN,<0.001,<0.1,<0.0005,12,<5,<0.02,<0.01,<0.1,<0.06,<0.003,1,<0.005,<0.0004,<0.01,6.7,<0.005,<0.01,11.7,<5,54,35,<0.05


In [12]:
from platform import python_version

print(python_version())

3.9.13
